In [2]:
import pandas as pd
import numpy as np
from numpy.random import RandomState
from numpy.linalg import cholesky

from numpy_sugar.linalg import economic_qs_linear, economic_qs

from cellregmap import run_association, run_interaction, estimate_betas


import gxslmm as gs

In [3]:
random = RandomState(1)
n = 50                               # number of samples (cells)
p = 5                                # number of individuals
k = 4                                # number of contexts
y = random.randn(n, 1)               # outcome vector (expression phenotype, one gene only)
W = np.ones((n, 1))                     # intercept (covariate matrix)
hK = random.randn(n, p)              # decomposition of kinship matrix (K = hK @ hK.T)
random = RandomState(3)
hS = random.randn(n, p)              # decomposition of Spatial kernel (S = hS @ hS.T)
x = 1.0 * (random.rand(n, 1) < 0.2)  # SNP vector

In [4]:
hK

array([[ 3.00170320e-01, -3.52249846e-01, -1.14251820e+00,
        -3.49342722e-01, -2.08894233e-01],
       [ 5.86623191e-01,  8.38983414e-01,  9.31102081e-01,
         2.85587325e-01,  8.85141164e-01],
       [-7.54397941e-01,  1.25286816e+00,  5.12929820e-01,
        -2.98092835e-01,  4.88518147e-01],
       [-7.55717130e-02,  1.13162939e+00,  1.51981682e+00,
         2.18557541e+00, -1.39649634e+00],
       [-1.44411381e+00, -5.04465863e-01,  1.60037069e-01,
         8.76168921e-01,  3.15634947e-01],
       [-2.02220122e+00, -3.06204013e-01,  8.27974643e-01,
         2.30094735e-01,  7.62011180e-01],
       [-2.22328143e-01, -2.00758069e-01,  1.86561391e-01,
         4.10051647e-01,  1.98299720e-01],
       [ 1.19008646e-01, -6.70662286e-01,  3.77563786e-01,
         1.21821271e-01,  1.12948391e+00],
       [ 1.19891788e+00,  1.85156417e-01, -3.75284950e-01,
        -6.38730407e-01,  4.23494354e-01],
       [ 7.73400683e-02, -3.43853676e-01,  4.35968568e-02,
        -6.20000844e-01

In [5]:
S = hS @ hS.T
print(S.shape)

K = hK @ hK.T
print(K.shape)


(50, 50)
(50, 50)


In [6]:
K

array([[ 1.68520715, -1.46791472, -1.2517142 , ...,  0.22823683,
        -0.34304934, -0.92149098],
       [-1.46791472,  2.78000602,  1.43345428, ...,  0.05547911,
         0.03209465,  1.15212777],
       [-1.2517142 ,  1.43345428,  2.72940119, ...,  0.32466572,
         0.4837169 ,  1.72071916],
       ...,
       [ 0.22823683,  0.05547911,  0.32466572, ...,  1.32929681,
         0.80002312,  1.19728795],
       [-0.34304934,  0.03209465,  0.4837169 , ...,  0.80002312,
         0.75090037,  1.01896275],
       [-0.92149098,  1.15212777,  1.72071916, ...,  1.19728795,
         1.01896275,  2.13455849]])

In [7]:
new_hK = gs.buildchol(K) # K = hK @ hK.T
# new_hK = cholesky(K) # K = hK @ hK.T
new_hK @ new_hK.T

array([[ 1.68520715, -1.46791472, -1.2517142 , ...,  0.22823683,
        -0.34304934, -0.92149098],
       [-1.46791472,  2.78000602,  1.43345428, ...,  0.05547911,
         0.03209465,  1.15212777],
       [-1.2517142 ,  1.43345428,  2.72940119, ...,  0.32466572,
         0.4837169 ,  1.72071916],
       ...,
       [ 0.22823683,  0.05547911,  0.32466572, ...,  1.32929681,
         0.80002312,  1.19728795],
       [-0.34304934,  0.03209465,  0.4837169 , ...,  0.80002312,
         0.75090037,  1.01896275],
       [-0.92149098,  1.15212777,  1.72071916, ...,  1.19728795,
         1.01896275,  2.13455849]])

In [8]:
S

array([[ 6.9485937 , -0.51712397, -5.07848024, ...,  0.64785841,
        -0.68042035,  1.78495017],
       [-0.51712397,  0.75548702, -0.25846397, ...,  0.77110514,
        -1.35706127, -2.14146879],
       [-5.07848024, -0.25846397,  6.21066226, ..., -0.29019682,
         4.52690861,  1.71506242],
       ...,
       [ 0.64785841,  0.77110514, -0.29019682, ...,  2.02681143,
        -1.12109613, -0.33106667],
       [-0.68042035, -1.35706127,  4.52690861, ..., -1.12109613,
         8.8480685 ,  4.16575015],
       [ 1.78495017, -2.14146879,  1.71506242, ..., -0.33106667,
         4.16575015,  9.35408384]])

In [9]:
new_hS = gs.buildchol(S) # S = hS @ hS.T
new_hS @ new_hS.T

array([[ 6.9485937 , -0.51712397, -5.07848024, ...,  0.64785841,
        -0.68042035,  1.78495017],
       [-0.51712397,  0.75548702, -0.25846397, ...,  0.77110514,
        -1.35706127, -2.14146879],
       [-5.07848024, -0.25846397,  6.21066226, ..., -0.29019682,
         4.52690861,  1.71506242],
       ...,
       [ 0.64785841,  0.77110514, -0.29019682, ...,  2.02681143,
        -1.12109613, -0.33106667],
       [-0.68042035, -1.35706127,  4.52690861, ..., -1.12109613,
         8.8480685 ,  4.16575015],
       [ 1.78495017, -2.14146879,  1.71506242, ..., -0.33106667,
         4.16575015,  9.35408384]])

In [10]:
economic_qs_linear(new_hK)

((array([[ 7.26432820e-02,  4.83751985e-02,  1.70664512e-01,
          -4.53131493e-03, -5.56096793e-02],
         [-2.86521879e-01, -6.46455609e-02, -9.72251131e-02,
           9.01902017e-02,  4.91708132e-02],
         [-6.50448222e-02, -2.64035647e-01, -9.58360248e-02,
           1.78217156e-02,  7.93265538e-03],
         [ 1.86120897e-02, -1.31208349e-02, -1.58365314e-01,
          -4.01286400e-02,  3.45922369e-01],
         [ 1.44050449e-02, -2.10600471e-02, -1.14837793e-01,
          -2.29673296e-01, -4.33425233e-04],
         [ 1.44768339e-02, -1.29055497e-01, -2.44735896e-01,
          -2.02937678e-01, -6.24792384e-02],
         [-4.42764162e-02,  2.53360138e-02, -4.51475657e-02,
          -5.90206459e-02,  1.09693063e-02],
         [-2.04317121e-01,  8.95071747e-02, -8.25143695e-02,
          -3.42888343e-02, -7.68126823e-02],
         [-1.32071707e-01,  3.33093686e-02,  1.10461811e-01,
           1.47853622e-01, -4.98642751e-02],
         [-7.14951329e-02,  1.19541843e-02, -2

In [12]:
economic_qs(K)

((array([[ 7.26432820e-02,  4.83751985e-02,  1.70664512e-01,
          -4.53131493e-03, -5.56096793e-02],
         [-2.86521879e-01, -6.46455609e-02, -9.72251131e-02,
           9.01902017e-02,  4.91708132e-02],
         [-6.50448222e-02, -2.64035647e-01, -9.58360248e-02,
           1.78217156e-02,  7.93265538e-03],
         [ 1.86120897e-02, -1.31208349e-02, -1.58365314e-01,
          -4.01286400e-02,  3.45922369e-01],
         [ 1.44050449e-02, -2.10600471e-02, -1.14837793e-01,
          -2.29673296e-01, -4.33425233e-04],
         [ 1.44768339e-02, -1.29055497e-01, -2.44735896e-01,
          -2.02937678e-01, -6.24792384e-02],
         [-4.42764162e-02,  2.53360138e-02, -4.51475657e-02,
          -5.90206459e-02,  1.09693063e-02],
         [-2.04317121e-01,  8.95071747e-02, -8.25143695e-02,
          -3.42888343e-02, -7.68126823e-02],
         [-1.32071707e-01,  3.33093686e-02,  1.10461811e-01,
           1.47853622e-01, -4.98642751e-02],
         [-7.14951329e-02,  1.19541843e-02, -2

In [13]:
economic_qs_linear(new_hS)

((array([[ 0.02343075, -0.11024376, -0.13629944,  0.21039516,  0.20814053],
         [ 0.07826971,  0.02925485, -0.02249169, -0.08343096,  0.03679244],
         [-0.17530775,  0.07046707, -0.03862641, -0.11903118, -0.25637453],
         [ 0.2352536 , -0.05220072,  0.00832468, -0.23140938, -0.00206821],
         [-0.33941213,  0.02028233, -0.01300054, -0.14681986, -0.00740874],
         [-0.02216724,  0.13989921, -0.09812461, -0.01568614,  0.05208827],
         [ 0.22595694,  0.00085245, -0.32001871,  0.04363861,  0.03678535],
         [ 0.08163436,  0.20876747,  0.15214422, -0.29432207, -0.08151863],
         [-0.05803229,  0.28623842, -0.13625092, -0.06707957,  0.15955596],
         [-0.08624934,  0.14857023,  0.04245756,  0.019974  ,  0.27956022],
         [-0.07899246, -0.09278681,  0.01783438,  0.20050311, -0.23284758],
         [ 0.2116064 ,  0.12984449, -0.23899242, -0.23599296,  0.12689978],
         [ 0.10245739, -0.21183836, -0.17811967, -0.0344992 , -0.1991407 ],
         [ 0

In [14]:
economic_qs_linear(new_hS)

((array([[ 0.02343075, -0.11024376, -0.13629944,  0.21039516,  0.20814053],
         [ 0.07826971,  0.02925485, -0.02249169, -0.08343096,  0.03679244],
         [-0.17530775,  0.07046707, -0.03862641, -0.11903118, -0.25637453],
         [ 0.2352536 , -0.05220072,  0.00832468, -0.23140938, -0.00206821],
         [-0.33941213,  0.02028233, -0.01300054, -0.14681986, -0.00740874],
         [-0.02216724,  0.13989921, -0.09812461, -0.01568614,  0.05208827],
         [ 0.22595694,  0.00085245, -0.32001871,  0.04363861,  0.03678535],
         [ 0.08163436,  0.20876747,  0.15214422, -0.29432207, -0.08151863],
         [-0.05803229,  0.28623842, -0.13625092, -0.06707957,  0.15955596],
         [-0.08624934,  0.14857023,  0.04245756,  0.019974  ,  0.27956022],
         [-0.07899246, -0.09278681,  0.01783438,  0.20050311, -0.23284758],
         [ 0.2116064 ,  0.12984449, -0.23899242, -0.23599296,  0.12689978],
         [ 0.10245739, -0.21183836, -0.17811967, -0.0344992 , -0.1991407 ],
         [ 0

In [11]:
gs.gxslmm(y = y, x = x, W = W, S = new_hS, K = new_hK)

100%|██████████| 1/1 [00:00<00:00, 13.29it/s]


,p_association,p_interaction,rho1_association,e2_association,g2_association,eps2_association,rho1_interaction,e2_interaction,g2_interaction,eps2_interaction
0,1.0,0.834583,0.0,0.0,0.202924,0.079283,0.0,0.0,0.020776,0.951671
